In [27]:
#@title Carrega Bibliotecas

# Carrega Pacotes
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import datetime
import random
!pip install unidecode
from unidecode import unidecode
from sqlalchemy import create_engine 


In [28]:
#@title Scrap Sobrenomes

# Define Url de Sobrenomes
url = 'https://www.procob.com/os-sobrenomes-mais-comuns-do-brasil/'
 
# Conecta com URL
reqs = requests.get(url)
 
# Extrai informações de lista ordenada
soup = BeautifulSoup(reqs.text, 'lxml')
 
# Define Sobrenomes
Sobrenomes = np.random.choice([tag.text for tag in soup.find_all("ol")][0].split(), 90)

In [29]:
#@title Scrap Nome de Homens

# Define Url de Homens
url2 = 'https://www.minhavida.com.br/familia/materias/35919-100-nomes-para-meninos-mais-comuns-confira-lista'

# Conecta com URL
reqs = requests.get(url2)

# Extrai informações de lista ordenada
soup = BeautifulSoup(reqs.text, 'lxml')

# Define Sobrenomes
Homens = [tag.text for tag in soup.find_all("ul")][1].split()


In [30]:
#@title Scrap Nomes de Mulheres

# Define Url de Mulheres
url3 = 'https://www.minhavida.com.br/familia/materias/35925-100-nomes-para-meninas-mais-comuns-confira-lista'

# Conecta com URL
reqs = requests.get(url3)

# Extrai informações de lista ordenada
soup = BeautifulSoup(reqs.text, 'lxml')

# Define Sobrenomes
Mulheres = [tag.text for tag in soup.find_all("ul")][1].split()


In [31]:
#@title Scrap Nomes de Produtos

# Define Url de Melhores 10 celulares
url4 = 'https://mobizoo.com.br/opiniao/celulares-mais-vendidos/'

# Conecta com URL
reqs = requests.get(url4)

# Extrai informações da url
soup = BeautifulSoup(reqs.text, 'lxml')

# Define Produtos de Celulares
Produtos_Celulares = [tag.text for tag in soup.find_all("th")][5::4]

# Define Url de Melhores Desktops
url5 = 'https://spy.com/articles/gadgets/electronics/best-desktop-computers-reviews-247841/'

# Conecta com URL
reqs = requests.get(url5)

# Extrai informações da url
soup = BeautifulSoup(reqs.text, 'lxml')

# Define Variavel auxiliar com titulos completos com categoria
Produtos_Desktop = [tag.text for tag in soup.find_all("h2")][:11]

# Define Produtos Desktops 
Produtos_Desktop = list(map(lambda w: w.split(". ", 1)[1], Produtos_Desktop))

# Concatena Produtos
Produtos = Produtos_Celulares + Produtos_Desktop


In [32]:
#@title Gera emails e datas

# Conding
encoding = "utf-8"

# Declara email de Clientes
email_Cliente = [unidecode(i) + j for i, j in zip(list(Homens[0:89]), np.random.choice(["@gmail.com","@yahoo.com","@bh.com"], 90).tolist())] 

# Email de Fornecedor
email_Fornecedor = [unidecode(i) + j for i, j in zip(list(Homens[90:99]), np.random.choice(["@gmail.com","@yahoo.com","@bh.com"], 90).tolist())]

# Data de Inicio
inicio = datetime.date(2020, 1, 1)
fim = datetime.date(2020, 10, 1)

# Calcula Data
tempo_entre_datas = fim - inicio
dias_entre_datas = tempo_entre_datas.days
dias_aleatorios = random.randrange(dias_entre_datas)
data_aleatoria = inicio + datetime.timedelta(days=dias_aleatorios)

# Data Ordem
Data_Ordem = pd.date_range(start = '2020-01-01', end = '2020-10-01', periods=5000)

# Data Entrega
Data_Entrega = [data + datetime.timedelta(days=random.randrange(dias_entre_datas)) for data in Data_Ordem]



In [33]:
#@title Declara tabelas (dataframes)

# Declara data frame de Clientes com 500 clientes
Cliente = pd.DataFrame({'ID_Cliente' : range(89),
                       'Nome' : np.random.choice(Homens + Mulheres, 89).tolist(),
                       'Email' : email_Cliente})


# Declara data frame de Fornecedor com 50 Fornecedores
Fornecedor = pd.DataFrame({'ID_Fornecedor' : range(9),
                           'Nome' : Homens[90:99],
                           'Email' : email_Fornecedor})

# Declara data frame de Ordem com 5000 ordens
Ordem = pd.DataFrame({'ID_Ordem' : range(5000),
                      'ID_Cliente' : np.random.choice(range(500), 5000).tolist(),
                      'ID_Produto' : np.random.choice(range(20), 5000).tolist(),
                      'Data' : Data_Ordem})

# Remove Horario da Coluna Data tablea Ordem
Ordem['Data'] = [str(i) for i in pd.to_datetime(Ordem['Data']).dt.date]

# Declara dataframe de Produto com 10 produtos
Produto = pd.DataFrame({'ID_Produto' : range(21),
                        'ID_Fornecedor' : np.random.choice(range(9), 21).tolist(),
                        'Nome' : Produtos })

# Declara data frame de entrega com 5000 entregas
Entrega = pd.DataFrame({'ID_Entrega' : range(5000),
                        'ID_Produto' : Ordem.ID_Produto,
                        'Data' : Data_Entrega})

# Remove Horario da Coluna Data tabela Entrega
Entrega['Data'] = [str(i) for i in pd.to_datetime(Entrega['Data']).dt.date]

# Declara Data Frame Sobrenome
Sobrenome = pd.DataFrame({'ID_Cliente' : range(90),
                          'Sobrenome' : Sobrenomes})


In [34]:
#@title Define Funcao para converter panda em string SQL e cria motor

# Define Funcao
def para_sql(df, nome):
  rows = df.to_records(index=False)
  values = ', '.join(map(str, rows))
  sql = "INSERT INTO "+ nome + " VALUES {}".format(values)
  return sql.replace("""\'""", """\"""")

# Cria Motor de SQL 
engine = create_engine('sqlite:///ibdtp.db',  
                       echo = False) 
  

In [35]:
#@title Transforma Cliente em sql

# Converte para SQL
sql = para_sql(Cliente,"Cliente")

# Apaga tabela se já existir
engine.execute("DROP TABLE IF EXISTS Cliente;")
  
# Gera tabela 
engine.execute("CREATE TABLE Cliente ( \
                ID_Cliente mediumint(8) NOT NULL,\
                Nome varchar(255) default NULL,\
                Email varchar(255) default NULL,\
                PRIMARY KEY (ID_Cliente) \
                );")

# Popula tabela
engine.execute(str(sql))


In [36]:
#@title Transforma Sobrenome em sql

# Converte para SQL
sql = para_sql(Sobrenome,"Sobrenome")

# Apaga tabela se já existir
engine.execute("DROP TABLE IF EXISTS Sobrenome;")
  
# Gera tabela 
engine.execute("CREATE TABLE Sobrenome ( \
                ID_Cliente mediumint(8) NOT NULL, \
                Sobrenome varchar(255) default NULL, \
                PRIMARY KEY (ID_Cliente) \
                );")

# Popula tabela
engine.execute(str(sql))


In [37]:
#@title Transforma Fornecedor em sql

# Converte para SQL
sql = para_sql(Fornecedor,"Fornecedor")

# Apaga tabela se já existir
engine.execute("DROP TABLE IF EXISTS Fornecedor;")
  
# show the complete data 
engine.execute("CREATE TABLE Fornecedor ( \
                ID_Fornecedor  mediumint(8) NOT NULL, \
                Nome varchar(255) default NULL, \
                Email varchar(255) default NULL, \
                PRIMARY KEY (ID_Fornecedor) \
              );")

# Popula tabela de clientes
engine.execute(str(sql))


In [38]:
#@title Transforma Produto em sql

# Converte para SQL
sql = para_sql(Produto,"Produto")

# Apaga tabela se já existir
engine.execute("DROP TABLE IF EXISTS Produto;")
  
# show the complete data 
engine.execute("CREATE TABLE Produto ( \
                ID_Produto mediumint(8) NOT NULL, \
                ID_Fornecedor mediumint, \
                Nome varchar(255) default NULL, \
                PRIMARY KEY (ID_Produto) \
              );")

# Popula tabela de clientes
engine.execute(str(sql)) 


In [39]:
#@title Transforma Ordem em sql

# Converte para SQL
sql = para_sql(Ordem,"Ordem")

# Apaga tabela se já existir
engine.execute("DROP TABLE IF EXISTS Ordem;")
  
# show the complete data 
engine.execute("CREATE TABLE Ordem ( \
                ID_Ordem  mediumint(8) NOT NULL, \
                ID_Cliente mediumint, \
                ID_Produto mediumint, \
                Data varchar(255) default NULL, \
                PRIMARY KEY (ID_Ordem) \
                );")

# Popula tabela de clientes
engine.execute(str(sql))


In [40]:
#@title Transforma Entrega em sql

# Converte para SQL
sql = para_sql(Entrega,"Entrega")

# Apaga tabela se já existir
engine.execute("DROP TABLE IF EXISTS Entrega;")
  
# show the complete data 
engine.execute("CREATE TABLE Entrega ( \
                ID_Entrega mediumint(8) NOT NULL, \
                ID_Produto mediumint, \
                Data varchar(255), \
                PRIMARY KEY (ID_Entrega) \
                );")

# Popula tabela de clientes
engine.execute(str(sql))


In [41]:
#@title Consulta 1) π(IDCliente,   Nome,   Email) (Cliente)

# Avalia tempo gasto na consulta para projetar todos os valores da Tabela Cliente
%timeit pd.read_sql('SELECT * FROM Cliente', con = engine)


100 loops, best of 3: 2.12 ms per loop


In [42]:
#@title Consulta 2) π_(IDProduto,   IDFornecedor,   Nome) (Produto)

# Avalia tempo gasto na consulta para projetar produto
%timeit pd.read_sql('SELECT * FROM Produto', con = engine) 


100 loops, best of 3: 1.87 ms per loop


In [43]:
#@title Consulta 3) Cliente  ⋈_(IDCliente=IDCliente)  Ordem

# Declara query 3
q3 = "SELECT c.ID_Cliente, c.Nome, c.Email, o.ID_Ordem, o.ID_Produto, o.Data \
      FROM Cliente as c \
      JOIN Ordem as o ON c.ID_Cliente = o.ID_Cliente"

# Avalia tempo gasto na consulta para projetar a jução de Cliente e Ordem
%timeit pd.read_sql(q3, con = engine)


100 loops, best of 3: 6.23 ms per loop


In [44]:
#@title Consulta 4) Produto  ⋈_(IDProduto=IDProduto)  Ordem

# Declara Query 4
q4 = "SELECT p.ID_Produto, p.ID_Fornecedor, p.Nome as Nome_Produto, o.ID_Ordem, o.ID_Cliente, o.Data \
      FROM Produto as p \
      JOIN Ordem as o ON p.ID_Produto = o.ID_Produto"

# Avalia tempo gasto Consulta para projetar a jução de Produto e Ordem
%timeit pd.read_sql(q4, con = engine)


10 loops, best of 3: 20.7 ms per loop


In [45]:
#@title Consulta 5) Fornecedor  ⋈_(IDFornecedor=IDFornecedor)  Produto

# Declara query 5 
q5 = "SELECT f.ID_Fornecedor, f.Nome, f.Email, p.ID_Produto, p.Nome as Nome_Produto  \
      FROM Fornecedor as f \
      JOIN Produto as p ON f.ID_Fornecedor = p.ID_Fornecedor"

# Avalia tempo gasto na consulta para projetar a junção de Fornecedor e Produto
%timeit pd.read_sql(q5, con = engine)


100 loops, best of 3: 2.32 ms per loop


In [46]:
#@title Consulta 6) (Fornecedor  ⋈_(IDFornecedor=IDFornecedor)  Produto)⋈_(IDProduto=IDProduto)  Ordem

# Declara query 6
q6 = "SELECT f.ID_Fornecedor, f.Nome, f.Email, p.ID_Produto, p.Nome as Nome_Produto, o.ID_Cliente, o.Data \
      FROM Fornecedor as f \
      JOIN Produto as p ON f.ID_Fornecedor = p.ID_Fornecedor \
      JOIN Ordem as o ON p.ID_Produto = o.ID_Produto"

# Consulta para projetar a junção de Fornecedor, Produto e Ordem
%timeit pd.read_sql(q6, con = engine)


10 loops, best of 3: 23.3 ms per loop


In [47]:
#@title Consulta 7) (Produto  ⋈_(IDProduto=IDProduto)  Ordem)  ⋈_(IDCliente=IDCliente)  Cliente

# Declara query 7
q7 = "SELECT p.ID_Produto, p.ID_Fornecedor, c.Nome as Nome_Cliente, o.ID_Ordem, o.ID_Cliente, o.Data \
      FROM Produto as p \
      JOIN Ordem as o ON p.ID_Produto = o.ID_Produto \
      JOIN Cliente as c ON o.ID_Cliente = c.ID_Cliente"

# Avalia tempo gasto na consulta para projetar a jução de Produto, Ordem e Cliente
%timeit pd.read_sql(q4, con = engine)


100 loops, best of 3: 19.7 ms per loop


In [48]:
#@title Consulta 8) (Produto  ⋈_(IDProduto=IDProduto)  Ordem)  ⋈_(IDFornecedor=IDFornecedor)  Fornecedor

# Declara query 8
q8 = "SELECT p.ID_Produto, p.ID_Fornecedor, f.Nome as Nome_Fornecedor, o.ID_Ordem, o.ID_Cliente, o.Data \
      FROM Produto as p \
      JOIN Ordem as o ON p.ID_Produto = o.ID_Produto \
      JOIN Fornecedor as f ON p.ID_Fornecedor = p.ID_Fornecedor"

# Avalia tempo gasto na consulta para projetar a jução de Produto, Ordem e Fornecedor
%timeit pd.read_sql(q8, con = engine)


10 loops, best of 3: 150 ms per loop


In [49]:
#@title Consulta 9) J_(COUNT IDCliente) (Cliente  ⋈_(IDCliente=IDCliente)  Ordem) 

# Declara query 9
q9 = "SELECT c.ID_Cliente, c.Nome, COUNT(o.ID_Ordem) as Quantidade_de_Ordens  \
                         FROM Cliente as c \
                         JOIN Ordem as o ON c.ID_Cliente = o.ID_Cliente \
                         GROUP BY c.ID_Cliente \
                         ORDER BY Quantidade_de_Ordens DESC"

# Avalia tempo gasto na consulta para calcular a quantidade de ordens por cliente
%timeit pd.read_sql(q9, con = engine)


100 loops, best of 3: 3.51 ms per loop


In [50]:
#@title Consulta 10) J_(COUNT IDProduto) (Fornecedor  ⋈_(IDFornecedor=IDFornecedor)  Produto)

# Declara query 10
q10 = "SELECT f.ID_Fornecedor, f.Nome as Nome_Fornecedor, COUNT(p.ID_Produto) as Quantidade_de_Produtos \
       FROM Fornecedor as f \
       JOIN Produto as p ON f.ID_Fornecedor = p.ID_Fornecedor \
       GROUP BY f.ID_Fornecedor \
       ORDER BY Quantidade_de_Produtos DESC"

# Consulta para Calcular a quantidade de produtos por fornecedor
%timeit pd.read_sql(q10, con = engine)


1000 loops, best of 3: 1.96 ms per loop
